Construccion de sistema de recomendaciones de anime.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
anime = pd.read_csv("https://bit.ly/2kiJkrW")
# Saber la dimension de nuestro conjunto
print(anime.shape)
anime.head()

(12294, 7)


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


Con estos datos se pueden hacer predicciones

In [5]:
# Regresa una matriz si existe numeros
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

rating tiene $230$ nulos y asi suvecesivamente.

In [15]:
known_animes = {"Naruto Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}
# Actualizar capitulos de esos animes
for k, v in known_animes.items():
    anime.loc[anime['name'] == k, 'episodes'] == v

# Localiza un renglon
anime.loc[(anime['genre'] == 'Hentai') & (anime['episodes'] == 'Unknown'),
          'episodes'] = '1'

anime.loc[(anime['type'] == 'OVA') & (anime['episodes'] == 'Unknown'),
          'episodes'] = '1'

anime.loc[(anime['type'] == 'Movie') & (anime['episodes'] == 'Unknown'),
          'episodes'] = '1'

# Tambien podemos utilizar la notacion dot porque es un data frame anime.episodes
anime['episodes'] = anime['episodes'].map(lambda x: np.nan if x == 'Unknown' else x)
# Utilizamos la mediana para encontrar el punto medio de los datos
anime['episodes'].fillna(anime['episodes'].median(), inplace=True)


In [22]:

pd.get_dummies(anime['type']).head(3)

,Movie,Music,ONA,OVA,Special,TV
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1


Para el type en los encabezados por eso ponemos doble `[['type']]`

In [21]:
pd.get_dummies(anime[['type']]).head(3)

,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1


In [25]:
# Llenamos los demas valores faltantes
anime['members'] = anime.members.astype(float)
anime.rating = anime['rating'].astype(float)
# inplace lo que hace es hacerlo en el mismo conjunto de datos
anime.rating.fillna(anime.rating.median(), inplace=True)

In [33]:
# dumies de los genermos y agregamos columnas
anime_features = pd.concat([anime['genre'].str.get_dummies(sep=','),
                            pd.get_dummies(anime[['type']]),
                            anime['rating'], anime['members'],
                            anime['episodes']], axis=1)

In [34]:
anime_features

,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,...,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV,rating,members,episodes
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,9.37,200630.0,1
1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.26,793665.0,64
2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.25,114262.0,51
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.17,673572.0,24
4,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.16,151266.0,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.15,211.0,1
12290,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.28,183.0,1
12291,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.88,219.0,4
12292,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.98,175.0,1


## Escalamiento min-max

Tipos de escalmientos:
- min-max
- normalizacion
- estandarizacion

In [36]:
from sklearn.preprocessing import MinMaxScaler

min_max = MinMaxScaler()
anime_features = min_max.fit_transform(anime_features)

In [37]:
np.round(anime_features, 2)

array([[0.  , 0.  , 0.  , ..., 0.92, 0.2 , 0.  ],
       [1.  , 0.  , 0.  , ..., 0.91, 0.78, 0.03],
       [0.  , 0.  , 1.  , ..., 0.91, 0.11, 0.03],
       ...,
       [0.  , 0.  , 0.  , ..., 0.39, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.4 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.45, 0.  , 0.  ]])

Una vez acabado el procesamiento, ya podemos pasar a nuestro modelo.

In [39]:
# Modelo KNN
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
knn.fit(anime_features)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

- https://scikit-learn.org/stable/modules/neighbors.html

- https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree


Ahora quiero que nos recomiende unos animes

In [ ]:
# Obtener recomendaciones necesitamos funciones auxiliares
distances, indices = knn.kneighbors(anime_features)

In [41]:
all_anime_names = list(anime.name.values)


def get_index_from_name(name):
    return anime[anime['name'] == name].index.tolist()[0]

def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name, all_anime_names.index(name))

In [49]:
get_id_from_partial_name('Naruto')

Boruto: Naruto the Movie 486
Naruto: Shippuuden 615
The Last: Naruto the Movie 719
Naruto: Shippuuden Movie 6 - Road to Ninja 784
Naruto 841
Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi 1103
Naruto: Shippuuden Movie 5 - Blood Prison 1237
Naruto x UT 1343
Naruto: Shippuuden Movie 4 - The Lost Tower 1472
Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono 1573
Naruto: Shippuuden Movie 1 1827
Naruto: Shippuuden Movie 2 - Kizuna 1828
Naruto Shippuuden: Shippuu! &quot;Konoha Gakuen&quot; Den 2374
Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!! 2416
Naruto SD: Rock Lee no Seishun Full-Power Ninden 2457
Naruto Shippuuden: Sunny Side Battle 2458
Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shinobu Houjou Dattebayo! 2756
Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!! 2997
Naruto Movie 2: Dai Gekitotsu! Maboroshi no Chiteiiseki Dattebayo! 3449
Naruto: Dai Katsugeki!! Yuki Hime Shinobu Houjou Dattebayo! Special: Konoha Annual Sports Festival 3529
Naru

In [46]:
def print_similiar_animes(query=None, id=None):
    if id:
        for id in indices[id][1:]:
            print(anime.iloc[id]['name'])
        if query:
            found_id = get_index_from_name(query)
            for id in indices[found_id][1:]:
                print(anime.iloc[id]['name'])

In [48]:
print_similiar_animes('Naruto')